# Commercial Register Exploration

In [176]:
import requests
import fake_useragent
from datetime import datetime
from bs4 import BeautifulSoup, Comment

In [158]:
ua = fake_useragent.UserAgent()
ua_chrome = ua.chrome
print(ua_chrome)

Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.15 (KHTML, like Gecko) Ubuntu/10.10 Chromium/10.0.611.0 Chrome/10.0.611.0 Safari/534.15


## Get SessionID

In [161]:
url = "https://www.unternehmensregister.de/ureg/"

payload = {}

headers = {
  'Host': 'www.unternehmensregister.de',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
  'User-Agent': ua_chrome,
  'Accept-Language': 'en-GB,en;q=0.9'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)

200


In [162]:
# read token and javax state
import re

regex = r'jsessionid=([a-zA-Z0-9]+\.[a-zA-Z0-9]+-1)\b'


match = re.search(regex, response.text)
if match:
    jsessionid = match.group(1)
    print(jsessionid)
    

soup = BeautifulSoup(response.text, 'html.parser')

input_element = soup.find('input', {'name': 'javax.faces.ViewState'})
if input_element:
    viewstate = input_element['value']
    print(viewstate)

6B4DA279666C52A8A94D68F610A69388.web01-1
5205236914724907181:-2556195301656197106


## Search for Company

In [163]:
import requests

search_term = "Tacto"
url = f"https://www.unternehmensregister.de/ureg/index.html;jsessionid={jsessionid}"

payload = f'globalSearchForm=globalSearchForm&globalSearchForm%3AextendedResearchCompanyName={search_term}&globalSearchForm%3Ahidden_element_used_for_validation_only=aaa&submitaction=globalsearch&globalsearch=true&globalSearchForm%3AbtnExecuteSearchOld=Suchen&javax.faces.ViewState={viewstate}'
headers = {
  'Host': 'www.unternehmensregister.de',
  'Origin': 'https://www.unternehmensregister.de',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
  'User-Agent': ua_chrome,
  'Referer': 'https://www.unternehmensregister.de/ureg/',
  'Accept-Language': 'en-GB,en;q=0.9',
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [164]:
url = f"https://www.unternehmensregister.de/ureg/result.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'Host': 'www.unternehmensregister.de',
  'Cache-Control': 'max-age=0',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Referer': f'https://www.unternehmensregister.de/ureg/index.html;jsessionid={jsessionid}',
  'Accept-Language': 'en-DE,en;q=0.9,de-DE;q=0.8,de;q=0.7,da-DK;q=0.6,da;q=0.5,en-US;q=0.4'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [165]:
# extract data
from bs4 import BeautifulSoup

html = response.text
soup = BeautifulSoup(html, "html.parser")

# Extract company name
company_name = soup.find("div", class_="company_result").find("b").get_text(strip=True)

# Assuming 'soup' is your BeautifulSoup object
company_link = soup.find("a", string="Registerinformationen des Registergerichts")["href"]

# Use regex to find company number
company_number = re.search(r'company=(\d+)', company_link)

# Check if a match was found
if company_number:
    company_id = company_number.group(1)  # Group 1 contains the first set of parentheses matches
else:
    company_id = None


print(company_name)
print(company_id)

Tacto Technology GmbH
4752128


## Load Register Information

In [166]:
import requests

url = f"https://www.unternehmensregister.de/ureg/registerPortalAdvice.html;jsessionid={jsessionid}?submitaction=regnav&company={company_id}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


## Loop Through Document Tree to find Gesellschafterliste


In [167]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}?submitaction=showDkTree&searchIdx=0"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [168]:
soup = BeautifulSoup(response.text, 'html.parser')

for a in soup.find_all('a'):
    i = a.find('i', title="Öffnet diesen Baum des Dokumentenbaums")
    if i is not None:
        href_value = a['href']
        break
else:
    href_value = None

submit_action = href_value
print(submit_action)

?submitaction=showDkTree&gkz=D2601&gkzalt=&rtype=HRB&rnr=257852&action=tree&id=N__0_0&state=2&addedId=N__0_0&searchIdx=0


In [169]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)


200


In [170]:
soup = BeautifulSoup(response.text, 'html.parser')

tag_a = soup.find('a', string="Liste der Gesellschafter")
href_value = tag_a['href'] if tag_a else None
submit_action = href_value
print(submit_action)

?submitaction=showDkTree&action=select&addedId=N__0_0_1&searchIdx=0


In [171]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)

200


In [172]:
soup = BeautifulSoup(response.text, 'html.parser')

date_format = "%d.%m.%Y"

# Initialize the latest_date and the href_value
latest_date = datetime.min
href_value = ""

# Find all the 'a' tags with "Liste der Gesellschafter"
tags_a = soup.find_all('a', string=lambda x: x and "Liste der Gesellschafter" in x)

# Loop through all the tags
for tag_a in tags_a:
    # find date in the format dd.mm.yyyy
    date_string = re.search(r'\d{2}\.\d{2}\.\d{4}', tag_a.text)
    if date_string:
        date = datetime.strptime(date_string.group(), date_format)
        if date > latest_date:
            latest_date = date
            href_value = tag_a['href']

print(latest_date)
print(href_value)

submit_action = href_value




2022-11-18 00:00:00
?submitaction=showDkTree&action=select&addedId=N__0_0_1_0&searchIdx=0


In [173]:
url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}{submit_action}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)

200


## Add Document to Warenkorb

In [178]:
print(response.text)

<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [179]:
soup = BeautifulSoup(response.text, 'html.parser')
tag_td = soup.find('td', class_='submit-container')

inputs = {}

# Find all 'input' tags and store their 'id' and 'value' as key-value pairs in the dictionary
for input_tag in soup.find_all('input'):
    inputs[input_tag.get('id')] = input_tag.get('value')

# Now you can access each input value as a variable using its id. For example, to get the value of 'submitaction':
submitaction = inputs.get('submitaction')

# Print the variable
print(inputs)


{None: 'Suchen', 'loginForm:username': None, 'loginForm:password': '', 'loginForm:btnLogin': 'Anmelden', 'j_id1:javax.faces.ViewState:0': '6547407444316429241:-2340537418027087394', 'searchRegisterForm:registerDataCompanyName': 'Tacto Technology GmbH', 'searchRegisterForm:registerDataSearchEntrysCheck': None, 'searchRegisterForm:registerDataCompanyLocation': '', 'searchRegisterForm:registerDataRegisterNumber': '257852', 'searchRegisterForm:searchFederalState1': None, 'searchRegisterForm:searchFederalState2': None, 'searchRegisterForm:searchFederalState3': None, 'searchRegisterForm:searchFederalState4': None, 'searchRegisterForm:searchFederalState5': None, 'searchRegisterForm:searchFederalState6': None, 'searchRegisterForm:searchFederalState7': None, 'searchRegisterForm:searchFederalState8': None, 'searchRegisterForm:searchFederalState9': None, 'searchRegisterForm:searchFederalState10': None, 'searchRegisterForm:searchFederalState11': None, 'searchRegisterForm:searchFederalState12': Non

In [95]:


url = f"https://www.unternehmensregister.de/ureg/registerPortal.html;jsessionid={jsessionid}"

payload = f'ausdruckart=DK_ID&cname=Tanso%20Technologies%20GmbH&docnames=N__0_0_1_0&format=pdf&gkz=D2601&gkzalt=&hasChildren=true&iddok=addedId&location=&rnr=269123&rtype=HRB&searchIdx=0&state=2&submitaction=ajax_check4duplicate'
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'Accept': '*/*',
  'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
  'X-Requested-With': 'XMLHttpRequest',
  'sec-ch-ua-mobile': '?0',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'sec-ch-ua-platform': '"macOS"',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Dest': 'empty',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


200


In [131]:
soup = BeautifulSoup(response.text, 'html.parser')

tag_a = soup.find('a', string="Liste der Gesellschafter")
href_value = tag_a['href'] if tag_a else None

submit_action = href_value.split('submitaction=')[1]
print(submit_action)

showDkTree&action=select&addedId=N__0_0_1&searchIdx=0


In [96]:
url = f"https://www.unternehmensregister.de/ureg/doccart.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [82]:
url = f"https://www.unternehmensregister.de/ureg/payment_overview.html;jsessionid={jsessionid}"

payload = {}
headers = {
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.status_code)



200


In [83]:
soup = BeautifulSoup(response.text, 'html.parser')
viewstate_input = soup.find('input', {'name': 'javax.faces.ViewState'})
viewstate = viewstate_input['value'] if viewstate_input else None

print(viewstate)


-6223541390404312379:7277534571971159532


In [84]:
url = f"https://www.unternehmensregister.de/ureg/payment_overview.html;jsessionid={jsessionid}"

payload = f'javax.faces.ViewState={viewstate}&paymentFormOverview=paymentFormOverview&paymentFormOverview%3AacceptAGB=true&paymentFormOverview%3AbtnNext=Jetzt%20freischalten&submitAmount=0&submitaction=paymentSubmit'
headers = {
  'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"macOS"',
  'Upgrade-Insecure-Requests': '1',
  'Content-Type': 'application/x-www-form-urlencoded',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-User': '?1',
  'Sec-Fetch-Dest': 'document',
  'Cookie': 'checkCookieSession=true'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.status_code)
print(response.text)



500
<!DOCTYPE html
  PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/transitional.dtd">
<html>
   <head>
      <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
      <title>Unternehmensregister</title>
      <style type="text/css">
      html,
      body {
        font-family: "HelveticaNeue-Light", "Helvetica Neue Light", "Helvetica Neue", Helvetica, Arial, "Lucida Grande", sans-serif;
        font-weight: 300;
        font-weight: bold;
        height: 100%;
        width: 100%;
      }
      #container {
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        position: fixed;
        display: table;
      }
      .inner {
        display: table-cell;
        text-align: center;
        vertical-align: middle;
        width: 15em;
      }
      p {

      }
      </style>
   </head>
   <body>
    <div id="container">
      <div class="inner">
        <h2>Fehler</h2>
        <p>
          Dieser Dienst ist mome